In [1]:
!pip install xgboost catboost polars optuna -q

In [2]:
!pip install scikit-learn-intelex -q

# Importing required libraries

In [3]:
import pandas as pd
import polars as pl
import optuna
import pickle
from sklearn.model_selection import cross_val_score
from scipy.stats import ttest_rel

import numpy as np
## Enabling intel optimizations to 
import matplotlib.pyplot as plt
from sklearnex import patch_sklearn
patch_sklearn()

Matplotlib is building the font cache; this may take a moment.
Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,f1_score, roc_auc_score, accuracy_score

# Helper functions

In [5]:
# Helper functions
def diag_med_lab_pid_exist_check(modeling_pids, diag_pid, medications_pid, lab_pid,age_data = None):
    # Convert sets of pids for faster lookup
    diag_pid_set = set(diag_pid)
    medications_pid_set = set(medications_pid)
    lab_pid_set = set(lab_pid)

    # Create the result list using a single loop
    if age_data:
        result = [
        f"{age}_{int(pid in diag_pid_set)}{int(pid in medications_pid_set)}{int(pid in lab_pid_set)}"
        for pid,age in zip(modeling_pids,age_data)
    ]
    else:
        result = [
            f"{int(pid in diag_pid_set)}{int(pid in medications_pid_set)}{int(pid in lab_pid_set)}"
            for pid in modeling_pids
        ]
    
    return result

In [6]:
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix

def get_metrics(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)

    roc_auc = roc_auc_score(y_test, y_pred_proba[:,1])
    print("AUC: ", roc_auc)

    report = classification_report(y_test, y_pred)
    print("Classification Report:\n", report)

    # Compute confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    TN, FP, FN, TP = cm.ravel()

    # Calculate Sensitivity and Specificity
    sensitivity = TP / (TP + FN) if (TP + FN) > 0 else 0  # handle division by zero
    specificity = TN / (TN + FP) if (TN + FP) > 0 else 0  # handle division by zero
    
    print("Sensitivity (Recall):", sensitivity)
    print("Specificity:", specificity)

# Config class

In [7]:
class ml_config:
    base_folder ='../Determine_final_modeling_datasets/'
    columns_to_ignore_cat = ['PATIENT_NUM','FirstOutcomeDate','Outcome']
    target_column = 'Outcome'
    z_bmi_cvs_file = 'Determine_joined_med_usage_lab_latest_diag_phe_with_icd10z_bmi_bp_cvs_ordinal_nominal_encoded.parquet'
    without_z_bmi_file = 'Determine_joined_med_usage_lab_latest_diag_phe_without_icd10z_bmi_bp__ordinal_nominal_encoded.parquet'
    with_z_bmi_file = 'Determine_joined_med_usage_lab_latest_diag_phe_with_icd10z_bmi_bp__ordinal_nominal_encoded.parquet'
    ignore_med_patients = 'pat_num_ignore_meds.pkl'
    
    drop_low_feature_pids = False
    

# model1_df = pl.scan_parquet(ml_config.base_folder + ml_config.z_bmi_cvs_file)
# model2_df = pl.scan_parquet(ml_config.base_folder + ml_config.without_z_bmi_file)
# model3_df = pl.scan_parquet(ml_config.base_folder + ml_config.with_z_bmi_file)
    
# print(len(model1_df.collect()))
# print(len(model2_df.collect()))
# print(len(model3_df.collect()))

In [8]:
with open("../Error_analysis_files/error_analysis_1_feature_pids.pkl",'rb') as f:
    pids_1feature = pickle.load(f)

with open("../Error_analysis_files/error_analysis_3_feature_pids.pkl",'rb') as f:
    pids_3feature = pickle.load(f)

In [9]:
with open(ml_config.ignore_med_patients,'rb') as f:
    ignore_med_pat = pickle.load(f)

# Loading modeling data file

In [ ]:
ml_config.file = ml_config.z_bmi_cvs_file

In [ ]:
modeling_df_1 = pl.read_parquet(ml_config.base_folder + ml_config.without_z_bmi_file)
modeling_df_2 = pl.read_parquet(ml_config.base_folder + ml_config.z_bmi_cvs_file)

In [ ]:
modeling_df_1 = modeling_df_1.filter(~pl.col('PATIENT_NUM').is_in(ignore_med_pat))
modeling_df_2 = modeling_df_2.filter(~pl.col('PATIENT_NUM').is_in(ignore_med_pat))

In [ ]:
np.unique(modeling_df_1['Outcome'].to_list(),return_counts = True)

In [ ]:
np.unique(modeling_df_2['Outcome'].to_list(),return_counts = True)

In [ ]:
# modeling_df  = modeling_df.drop(ml_config.columns_to_drop)
# modeling_df.head()

In [ ]:
### Defining categorical columns
cat_features = [col for col in modeling_df_1.columns if  not (col.startswith('LOINC') 
                                                            or col in ml_config.columns_to_ignore_cat
                                                            or col in ['BMI',
                                                                       'mode_height',
                                                                       'average_weight',
                                                                         'average_diastolic_value',
                                                                         'average_systolic_value',
                                                                          "ACS_MedHHIncome", 
                                                                       "ACS_GINI", 
                                                                       "ACS_Unemployment", 
                                                                       "ACS_pctPoverty100", 
                                                                       "ACS_pctCollGrad"]
                                                                          )]
cat_features[:10]

In [ ]:
loinc_columns = [col for col in modeling_df_1.columns if col.startswith('LOINC')]
len(loinc_columns)

In [ ]:
modeling_df_1 = modeling_df_1.with_columns([
    pl.col(col).cast(pl.Float32)
    for col in loinc_columns
])

modeling_df_2 = modeling_df_2.with_columns([
    pl.col(col).cast(pl.Float32)
    for col in loinc_columns
])

# Train/test split

In [ ]:
# saving test pids
# with open('test_data_pids.pkl', 'wb') as file: 
#     # A new file will be created 
#     pickle.dump(test_pids, file) 
    
# with open('train_data_pids.pkl', 'wb') as file: 
#     # A new file will be created 
#     pickle.dump(train_pids, file) 

# Open the file in binary mode 
with open('train_data_pids.pkl', 'rb') as file: 
    train_pids = pickle.load(file) 
with open('test_data_pids.pkl', 'rb') as file: 
    test_pids = pickle.load(file) 

In [ ]:
data_train_1 = modeling_df_1.filter(pl.col('PATIENT_NUM').is_in(train_pids))
data_test_1 = modeling_df_1.filter(pl.col('PATIENT_NUM').is_in(test_pids))


data_train_2 = modeling_df_2.filter(pl.col('PATIENT_NUM').is_in(train_pids))
data_test_2 = modeling_df_2.filter(pl.col('PATIENT_NUM').is_in(test_pids))

In [ ]:
np.unique(data_test_1['Outcome'].to_list(), return_counts =True)

In [ ]:
del modeling_df_1, modeling_df_2

In [ ]:
X_train_1, y_train_1 = data_train_1.drop(['PATIENT_NUM','FirstOutcomeDate','Outcome']).to_pandas(), data_train_1['Outcome'].to_pandas()
X_test_1, y_test_1 = data_test_1.drop(['PATIENT_NUM','FirstOutcomeDate','Outcome']).to_pandas(), data_test_1['Outcome'].to_pandas()

X_train_2, y_train_2 = data_train_2.drop(['PATIENT_NUM','FirstOutcomeDate','Outcome']).to_pandas(), data_train_2['Outcome'].to_pandas()
X_test_2, y_test_2 = data_test_2.drop(['PATIENT_NUM','FirstOutcomeDate','Outcome']).to_pandas(), data_test_2['Outcome'].to_pandas()

In [ ]:
type(y_train)

In [ ]:
del data_train_1, data_train_2

# T-tested

In [ ]:


# Assuming we have two trained models, model_1 and model_2
model_1 = CatBoostClassifier().load_model('../Determine_trained_models/Catboost_Determine_joined_med_usage_lab_latest_diag_phe_without_icd10z_bmi_bp__ordinal_nominal_encoded.parquet')
model_2 = CatBoostClassifier().load_model('../Determine_trained_models/Catboost_Determine_joined_med_usage_lab_latest_diag_phe_with_icd10z_bmi_bp_cvs_ordinal_nominal_encoded.parquet')

# Train the models
model_1.fit(X_train_1, y_train_1, verbose=0)
model_2.fit(X_train_2, y_train_2, verbose=0)

# Evaluate the models using cross-validation
scores_1 = cross_val_score(model_1, X_test_1, y_test_1, cv=5, scoring='roc_auc')
scores_2 = cross_val_score(model_2, X_test_2, y_test_2, cv=5, scoring='roc_auc')

# Calculate mean accuracy
mean_score_1 = np.mean(scores_1)
mean_score_2 = np.mean(scores_2)

print(f"Mean accuracy for feature set 1: {mean_score_1}")
print(f"Mean accuracy for feature set 2: {mean_score_2}")

# Perform paired t-test
t_stat, p_value = ttest_rel(scores_1, scores_2)

print(f"T-statistic: {t_stat}")
print(f"P-value: {p_value}")

# Interpretation
alpha = 0.05
if p_value <= alpha:
    print("The difference in performance is statistically significant.")
else:
    print("The difference in performance is not statistically significant.")

# Models comparision AUC curves

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier
import xgboost as xgb
import pickle

In [ ]:
catboost_model = CatBoostClassifier().load_model('../Determine_trained_models/Catboost_Determine_joined_med_usage_lab_latest_diag_phe_with_icd10z_bmi_bp_cvs_ordinal_nominal_encoded.parquet')
# xgboost_model = xgb.XGBClassifier().load_model('../Determine_trained_models/XGBoost_Determine_joined_med_usage_lab_latest_diag_phe_with_icd10z_bmi_bp_cvs_ordinal_nominal_encoded.parquet')
# with open('../Determine_trained_models/XGB_Determine_joined_med_usage_lab_latest_diag_phe_with_icd10z_bmi_bp_cvs_ordinal_nominal_encoded.pkl','rb') as f:
#     xgboost_model = pickle.load(f)
# with open('../Determine_trained_models/LR_Determine_joined_med_usage_lab_latest_diag_phe_with_icd10z_bmi_bp_cvs_ordinal_nominal_encoded.pkl','rb') as f:
#     lr_model = pickle.load(f)
# with open('../Determine_trained_models/RF_Determine_joined_med_usage_lab_latest_diag_phe_with_icd10z_bmi_bp_cvs_ordinal_nominal_encoded.pkl','rb') as f:
#     rf_model = pickle.load(f)

In [ ]:
#Load data
modeling_df = pl.read_parquet(ml_config.base_folder + ml_config.z_bmi_cvs_file)
modeling_df = modeling_df.filter(~pl.col('PATIENT_NUM').is_in(ignore_med_pat))

In [ ]:
modeling_df.head()

In [ ]:
loinc_columns = [col for col in modeling_df.columns if col.startswith('LOINC')]
len(loinc_columns)
modeling_df = modeling_df.with_columns([
    pl.col(col).cast(pl.Float32)
    for col in loinc_columns
])

In [ ]:
def collapse(list_gender):
    return np.argmax(list(list_gender.values()))

In [ ]:
modeling_df = modeling_df.with_columns(pl.struct(['Gender_CD_GQ','Gender_CD_M','Gender_CD_TG','Gender_CD_W']).map_elements(collapse, return_dtype = pl.Int8).alias('Sensitive_target'))


In [ ]:
with open('train_data_pids.pkl', 'rb') as file: 
    train_pids = pickle.load(file) 
with open('test_data_pids.pkl', 'rb') as file: 
    test_pids = pickle.load(file) 

In [ ]:
data_train = modeling_df.filter(pl.col('PATIENT_NUM').is_in(train_pids))
data_test = modeling_df.filter(pl.col('PATIENT_NUM').is_in(test_pids))

del modeling_df

In [ ]:
### Get the sensitive feature
GI_train_list = data_train['Sensitive_target'].to_list()
GI_test_list = data_test['Sensitive_target'].to_list()

In [ ]:
X_train, y_train = data_train.drop(['PATIENT_NUM','FirstOutcomeDate','Outcome','Sensitive_target']).to_pandas(), data_train['Outcome'].to_pandas()
X_test, y_test = data_test.drop(['PATIENT_NUM','FirstOutcomeDate','Outcome','Sensitive_target']).to_pandas(), data_test['Outcome'].to_pandas()


In [ ]:
del data_train, data_test

In [ ]:
from sklearn.metrics import roc_curve, auc

# Demogrpahic Parity Ratio

In [ ]:
!pip install fairlearn -q

In [ ]:
from fairlearn.datasets import fetch_adult
from fairlearn.postprocessing import ThresholdOptimizer, plot_threshold_optimizer
from fairlearn.metrics import demographic_parity_ratio, equalized_odds_ratio
from fairlearn.reductions import DemographicParity

In [ ]:
def demographic_parity_ratio(y_true, y_pred, sensitive_feature):
    # Convert to a DataFrame for easier handling
    results = pd.DataFrame({'y_true': y_true, 'y_pred': y_pred, 'sensitive': sensitive_feature})

    # Calculate positive rate for each subgroup
    positive_rates = results.groupby('sensitive').apply(
        lambda x: np.mean(x['y_pred'])
    )

    # Calculate demographic parity ratio
    dp_ratio = positive_rates.min() / positive_rates.max()

    return dp_ratio, positive_rates



catboost_pred = catboost_model.predict(X_test)

dp_ratio, positive_rates = demographic_parity_ratio(y_test, catboost_pred, GI_test_list)

print("Demographic Parity Ratio:", dp_ratio)
print("Positive Rates by Group:", positive_rates)

### t-test

In [ ]:
scores_1 = cross_val_score(catboost_model, X_test, y_test, cv=5, scoring='roc_auc')
scores_2 = cross_val_score(xgboost_model, X_test, y_test, cv=5, scoring='roc_auc')

# Calculate mean accuracy
mean_score_1 = np.mean(scores_1)
mean_score_2 = np.mean(scores_2)

print(f"Mean accuracy for feature set 1: {mean_score_1}")
print(f"Mean accuracy for feature set 2: {mean_score_2}")

# Perform paired t-test
t_stat, p_value = ttest_rel(scores_1, scores_2)

print(f"T-statistic: {t_stat}")
print(f"P-value: {p_value}")

# Interpretation
alpha = 0.05
if p_value <= alpha:
    print("The difference in performance is statistically significant.")
else:
    print("The difference in performance is not statistically significant.")

### t-test end

In [ ]:
catboost_prob = catboost_model.predict_proba(X_test)[:, 1]
xgboost_prob = xgboost_model.predict_proba(X_test)[:, 1]
rf_prob = rf_model.predict_proba(X_test)[:, 1]



scaler = StandardScaler()

# Fit and transform only the specified columns
X_train[loinc_columns] = scaler.fit_transform(X_train[loinc_columns])
X_test[loinc_columns] = scaler.transform(X_test[loinc_columns])
lr_prob = lr_model.predict_proba(X_test)[:, 1]

# Calculate ROC curves
fpr_catboost, tpr_catboost, _ = roc_curve(y_test, catboost_prob)
fpr_xgboost, tpr_xgboost, _ = roc_curve(y_test, xgboost_prob)
fpr_rf, tpr_rf, _ = roc_curve(y_test, rf_prob)
fpr_lr, tpr_lr, _ = roc_curve(y_test, lr_prob)

In [ ]:
# Plotting ROC curves
# Get predictions


# Calculate AUC
from sklearn.metrics import precision_recall_curve, auc
import matplotlib.pyplot as plt

# Calculate precision and recall
precision_catboost, recall_catboost, _ = precision_recall_curve(y_test, catboost_prob)
precision_xgboost, recall_xgboost, _ = precision_recall_curve(y_test, xgboost_prob)
precision_rf, recall_rf, _ = precision_recall_curve(y_test, rf_prob)
precision_lr, recall_lr, _ = precision_recall_curve(y_test, lr_prob)

# Calculate AUC for Precision-Recall
auc_pr_catboost = auc(recall_catboost, precision_catboost)
auc_pr_xgboost = auc(recall_xgboost, precision_xgboost)
auc_pr_rf = auc(recall_rf, precision_rf)
auc_pr_lr = auc(recall_lr, precision_lr)

# Plot Precision-Recall curves
plt.figure(figsize=(10, 8))
plt.plot(recall_catboost, precision_catboost, label=f'CatBoost (AUC-PR = {auc_pr_catboost:.3f})')
plt.plot(recall_xgboost, precision_xgboost, label=f'XGBoost (AUC-PR = {auc_pr_xgboost:.3f})')
plt.plot(recall_rf, precision_rf, label=f'Random Forest (AUC-PR = {auc_pr_rf:.3f})')
plt.plot(recall_lr, precision_lr, label=f'Logistic Regression (AUC-PR = {auc_pr_lr:.3f})')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve Comparison')
plt.legend(loc='lower left')
plt.grid()
plt.show()

# Models Confidence Interval AUC, precision and recall

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix
from sklearn.utils import resample

def calculate_confidence_interval(data, confidence=0.95):
    """Calculate the confidence interval for a given array of data."""
    n = len(data)
    mean = np.mean(data)
    std_err = np.std(data, ddof=1) / np.sqrt(n)
    margin_of_error = std_err * 1.96  # For 95% confidence level
    return mean, mean - margin_of_error, mean + margin_of_error

def get_metrics(y_true, probs, threshold=0.5):
    """Calculate AUC, sensitivity, specificity, PPV, and their confidence intervals."""
    auc_score, auc_low, auc_high = calculate_confidence_interval(resample_and_score(y_true, probs, roc_auc_score))

    # Calculate optimal threshold
    fpr, tpr, thresholds = roc_curve(y_true, probs)
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx]
    
    preds = (probs >= optimal_threshold).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true, preds).ravel()
    
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    ppv = tp / (tp + fp)

    sensitivity_scores = [score[0] for score in resample_and_score(y_true, probs, calculate_sensitivity_specificity, threshold=optimal_threshold)]
    specificity_scores = [score[1] for score in resample_and_score(y_true, probs, calculate_sensitivity_specificity, threshold=optimal_threshold)]
    ppv_scores = [score[2] for score in resample_and_score(y_true, probs, calculate_sensitivity_specificity, threshold=optimal_threshold)]
    
    sensitivity_mean, sens_low, sens_high = calculate_confidence_interval(sensitivity_scores)
    specificity_mean, spec_low, spec_high = calculate_confidence_interval(specificity_scores)
    ppv_mean, ppv_low, ppv_high = calculate_confidence_interval(ppv_scores)
    
    return (f"{auc_score:.4f} ({auc_low:.4f} - {auc_high:.4f})",
            f"{sensitivity_mean:.4f} ({sens_low:.4f} - {sens_high:.4f})",
            f"{specificity_mean:.4f} ({spec_low:.4f} - {spec_high:.4f})",
            f"{ppv_mean:.4f} ({ppv_low:.4f} - {ppv_high:.4f})")

def resample_and_score(y_true, probs, score_func, n_iterations=1000, **kwargs):
    scores = []
    for _ in range(n_iterations):
        y_resampled, prob_resampled = resample(y_true, probs)
        scores.append(score_func(y_resampled, prob_resampled, **kwargs))
    return scores

def calculate_sensitivity_specificity(y_true, probs, threshold):
    preds = (probs >= threshold).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true, preds).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    ppv = tp / (tp + fp)
    return sensitivity, specificity, ppv

# Calculate metrics for each model
metrics_catboost = get_metrics(y_test, catboost_prob)
metrics_xgboost = get_metrics(y_test, xgboost_prob)
metrics_lr = get_metrics(y_test, lr_prob)
metrics_rf = get_metrics(y_test, rf_prob)

# Create a DataFrame to display
df_metrics = pd.DataFrame({
    'Model': ['CatBoost', 'XGBoost', 'random forest','Logistic Regression'],
    'AUC ROC (CI)': [metrics_catboost[0], metrics_xgboost[0], metrics_rf[0], metrics_lr[0]],
    'Sensitivity (CI)': [metrics_catboost[1], metrics_xgboost[1], metrics_rf[1], metrics_lr[1]],
    'Specificity (CI)': [metrics_catboost[2], metrics_xgboost[2], metrics_rf[2], metrics_lr[2]],
    'PPV (CI)': [metrics_catboost[3], metrics_xgboost[3], metrics_rf[3], metrics_lr[3]],
})

print(df_metrics)

In [ ]:
df_metrics